In [1]:
!pip install -U git+https://github.com/faustomorales/keras-ocr.git
!pip install imgaug
!pip install -U opencv-python
%tensorflow_version 2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/faustomorales/keras-ocr.git to /tmp/pip-req-build-75was4aw
  Running command git clone -q https://github.com/faustomorales/keras-ocr.git /tmp/pip-req-build-75was4aw
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 603 kB 7.2 MB/s 
     |████████████████████████████████| 9.5 MB 55.7 MB/s 
     |████████████████████████████████| 930 kB 62.2 MB/s 
     |████████████████████████████████| 50 kB 8.3 MB/s 
  Created wheel for keras-ocr: filename=keras_ocr-0.9.0.post2+40bb9bd-py3-none-any.whl size=42441 sha256=395ed38f34cb967c37e2cf7ce5f322a5231d7070ae68addf74cc2ca4a5851340
  Stored in directory: /tmp/pip-ephem-wheel-cache-yo0535cs/wheels/20/d0/06/ed290995144ec8dde71d3a91d0d590145445c01be60ea70c9b
  Created wheel for validators: filename=validators-0.19

In [2]:
data_dir = '.'

import os
import math
import imgaug
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection
import tensorflow as tf

import keras_ocr
train_labels = keras_ocr.datasets.get_icdar_2013_detector_dataset(
    cache_dir='.',
    skip_illegible=False
)

Looking for ./icdar2013/Challenge2_Training_Task12_Images.zip
Looking for ./icdar2013/Challenge2_Training_Task2_GT.zip


In [3]:
train_labels, validation_labels = sklearn.model_selection.train_test_split(train_labels, train_size=0.8, random_state=42)

augmenter = imgaug.augmenters.Sequential([
    imgaug.augmenters.Affine(
      scale=(1.0, 1.2),
      rotate=(-5, 5)
    ),
    imgaug.augmenters.GaussianBlur(sigma=(0, 0.5)),
    imgaug.augmenters.Multiply((0.8, 1.2), per_channel=0.2)
])

generator_kwargs = {'width': 640, 'height': 640}
training_image_generator = keras_ocr.datasets.get_detector_image_generator(
    labels=train_labels,
    augmenter=augmenter,
    **generator_kwargs
)
validation_image_generator = keras_ocr.datasets.get_detector_image_generator(
    labels=validation_labels,
    **generator_kwargs
)

In [4]:
detector = keras_ocr.detection.Detector()
detector.model.compile(loss="mse", optimizer="adam", metrics=["acc"])

Looking for /root/.keras-ocr/craft_mlt_25k.h5


In [5]:
batch_size = 1
training_generator, validation_generator = [
    detector.get_batch_generator(
        image_generator=image_generator, batch_size=batch_size
    ) for image_generator in
    [training_image_generator, validation_image_generator]
]

In [6]:
detector.model.fit(
    x=training_generator,
    steps_per_epoch=math.ceil(len(train_labels) / batch_size),
    epochs=1000,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(restore_best_weights=True, patience=5),
        tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(data_dir, 'detector_icdar2013.h5'))
    ],
    validation_data=validation_generator,
    validation_steps=math.ceil(len(validation_labels) / batch_size),
    verbose=1
)

Epoch 1/1000
183/183 [==============================] - ETA: 0s - loss: 0.0145 - acc: 0.8874

/usr/local/lib/python3.7/dist-packages/keras_ocr/tools.py:580: RuntimeWarning: invalid value encountered in long_scalars
  rotation = np.arctan((tl[0] - bl[0]) / (tl[1] - bl[1]))


183/183 [==============================] - 87s 364ms/step - loss: 0.0145 - acc: 0.8874 - val_loss: 0.0139 - val_acc: 0.3470
Epoch 2/1000
183/183 [==============================] - 64s 353ms/step - loss: 0.0131 - acc: 0.9018 - val_loss: 0.0101 - val_acc: 0.9880
Epoch 3/1000
183/183 [==============================] - 66s 362ms/step - loss: 0.0112 - acc: 0.9822 - val_loss: 0.0579 - val_acc: 0.9884
Epoch 4/1000
183/183 [==============================] - 65s 354ms/step - loss: 0.0102 - acc: 0.9807 - val_loss: 0.0092 - val_acc: 0.9834
Epoch 5/1000
183/183 [==============================] - 65s 357ms/step - loss: 0.0099 - acc: 0.9812 - val_loss: 0.0222 - val_acc: 0.9832
Epoch 6/1000
183/183 [==============================] - 66s 361ms/step - loss: 0.0080 - acc: 0.8964 - val_loss: 0.0107 - val_acc: 0.9876
Epoch 7/1000
183/183 [==============================] - 65s 358ms/step - loss: 0.0066 - acc: 0.8165 - val_loss: 0.0069 - val_acc: 0.9923
Epoch 8/1000
183/183 [==============================] 

In [7]:
from google.colab import files
files.download('/content/detector_icdar2013.h5') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>